### Fisher Score- Chisquare Test For Feature Selection
Compute chi-squared stats between each non-negative feature and class.

This score should be used to evaluate categorical variables in a classification task.
This score can be used to select the n_features features with the highest values for the test chi-squared statistic from X, which must contain only non-negative features such as booleans or frequencies (e.g., term counts in document classification), relative to the classes.

Recall that the chi-square test measures dependence between stochastic variables, so using this function “weeds out” the features that are the most likely to be independent of class and therefore irrelevant for classification. The Chi Square statistic is commonly used for testing relationships between categorical variables.

It compares the observed distribution of the different classes of target Y among the different categories of the feature, against the expected distribution of the target classes, regardless of the feature categories.

In [1]:
import seaborn as sns
df = sns.load_dataset('titanic')

In [2]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [6]:
### [ "sex",'embarked','alone','pclass','survived']
df = df[[ "sex",'embarked','alone','pclass','survived']]

In [7]:
df.head()

,sex,embarked,alone,pclass,survived
0,male,S,False,3,0
1,female,C,False,1,1
2,female,S,True,3,1
3,female,S,False,1,1
4,male,S,True,3,0


In [8]:
##before applying the chi square test we need to perform label encoding

In [11]:
import numpy as np
### let's perform label encoding on sex column
df['sex']= np.where(df['sex']=='male',1,0)
##let's perform label emcoding on embark column
ordinal_label = {k: i for i, k in enumerate(df['embarked'].unique(), 0)}
df['embarked'] = df['embarked'].map(ordinal_label)

<ipython-input-11-2b096b817eaa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex']= np.where(df['sex']=='male',1,0)
<ipython-input-11-2b096b817eaa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embarked'] = df['embarked'].map(ordinal_label)


In [12]:
ordinal_label

{'S': 0, 'C': 1, 'Q': 2, nan: 3}

In [13]:
df.head()

,sex,embarked,alone,pclass,survived
0,0,0,False,3,0
1,0,1,False,1,1
2,0,0,True,3,1
3,0,0,False,1,1
4,0,0,True,3,0


In [15]:
##let's perform label emcoding on alone column
df['sex']= np.where(df['alone']==True,1,0)

<ipython-input-15-7f82bcf887ba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex']= np.where(df['alone']==True,1,0)


In [16]:
df.head()

,sex,embarked,alone,pclass,survived
0,0,0,False,3,0
1,0,1,False,1,1
2,1,0,True,3,1
3,0,0,False,1,1
4,1,0,True,3,0


In [17]:
### train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[[ "sex",'embarked','alone','pclass']],df["survived"],test_size=0.3,random_state= 0)

In [18]:
X_train

,sex,embarked,alone,pclass
857,1,0,True,1
52,0,1,False,1
386,0,0,False,3
124,0,0,False,1
578,0,1,False,3
...,...,...,...,...
835,0,1,False,1
192,0,0,False,3
629,1,2,True,3
559,0,0,False,3


In [19]:
X_test

,sex,embarked,alone,pclass
495,1,1,True,3
648,1,0,True,3
278,0,2,False,3
31,0,1,False,1
255,0,1,False,3
...,...,...,...,...
263,1,0,True,1
718,1,2,True,3
620,0,1,False,3
786,1,0,True,3


In [20]:
X_train.isnull().sum()

sex         0
embarked    0
alone       0
pclass      0
dtype: int64

In [22]:
## perform chisquare test
## chi2 return 2 values
##1.fscore and p value
from sklearn.feature_selection import chi2
f_p_values = chi2(X_train,y_train)

In [23]:
f_p_values

(array([ 9.03328564, 11.83961845,  9.03328564, 21.61080949]),
 array([2.65107556e-03, 5.79837058e-04, 2.65107556e-03, 3.33964360e-06]))

In [28]:
import pandas as pd
p_values = pd.Series(f_p_values[1])
p_values.index = X_train.columns
p_values.sort_index(ascending = False)

sex         0.002651
pclass      0.000003
embarked    0.000580
alone       0.002651
dtype: float64

### observation
sex column is the most important column when compared to the output feauture survivde columns